<a href="https://colab.research.google.com/github/KarenLira25/Proyecto1_AnalisisDatos/blob/main/Tarea3_Ajuste_parametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

El archivo que se adjunta ”Pueblapos.csv” son datos de los casos registrados(nuevos casos) para Puebla reportados por la secretar ́ıa de salud.
1.   Cargar los datos y Graficar
2.   Ajustar la primera ola con mínimos cuadrados.
3.   Ajustar la segunda ola con una cadena de Monte Carlo y dar intervalosde confianza para los parámetros.

**Nota 1:**  Puede usar un modelo SIR o si lo prefieren un SEIR. 

**Nota 2:**  Recordar que los datos del archivo son los nuevos casos y no corresponden directa-mente  a  los  Infectados  (I)  de  un  modelo  SIR.  Tienen  que  definir  una  nueva ecuación que sean los casos acumulados y transformar los datos en casos acumulados.  Es decir, la suma de los casos hasta el tiempo t.


In [10]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
file_name = '/content/drive/MyDrive/Puebla_pos.csv'
df  = pd.read_csv(file_name) 
df

,Unnamed: 0,FECHA_SINTOMAS,positivo
0,0,2020-03-02,1
1,1,2020-03-05,2
2,2,2020-03-08,3
3,3,2020-03-09,4
4,4,2020-03-10,3
...,...,...,...
673,673,2022-01-09,493
674,674,2022-01-10,556
675,675,2022-01-11,287
676,676,2022-01-12,149


In [12]:
from datetime import datetime
#Filrar los datos por años
#df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'], format='%Y-%m-%d')
#Año2020 = df.query("FECHA_SINTOMAS >= '2020-01-01' \
#                       and FECHA_SINTOMAS <= '2020-12-31'")
#print(Año2020)

#df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'], format='%Y-%m-%d')
#Año2021 = df.query("FECHA_SINTOMAS >= '2021-01-01' \
#                       and FECHA_SINTOMAS <= '2021-12-31'")
#print(Año2021)

#df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'], format='%Y-%m-%d')
#Año2022 = df.query("FECHA_SINTOMAS >= '2022-01-01' \
#                       and FECHA_SINTOMAS <= '2022-12-31'")
#print(Año2022)



In [13]:
#convertir los datos a tipo tiempo
from datetime import datetime
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'], format='%Y-%m-%d')

In [14]:
df=df.set_index('FECHA_SINTOMAS')
df['2020-03']

<ipython-input-14-dbf51bceaa10>:2: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df['2020-03']


,Unnamed: 0,positivo
FECHA_SINTOMAS,,
2020-03-02,0,1
2020-03-05,1,2
2020-03-08,2,3
2020-03-09,3,4
2020-03-10,4,3
2020-03-12,5,3
2020-03-13,6,1
2020-03-14,7,4
2020-03-15,8,5
